<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparação-do-Ambiente" data-toc-modified-id="Preparação-do-Ambiente-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparação do Ambiente</a></span></li><li><span><a href="#Conexão-ao-Banco-de-Dados" data-toc-modified-id="Conexão-ao-Banco-de-Dados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Conexão ao Banco de Dados</a></span></li></ul></div>

Uma das mais poderosas funcionalidades proporcionadas pela linguagem Python é justamente a facilidade em se trabalhar com dados. Em situações genéricas, o pool de conexões a bancos de dados também abre um leque gigantesco de possibilidades, permitindo assim com que usuários gerenciem, consultem e automatizem tarefas envolvendo persistências de dados utilizando diferentes SGBDs.

Neste cenário, considerando o desenvolvimento prévio composto pelo notebook de documentação da API `nba_api`, bem como a construção das classes responsáveis por consumir tal pacote python e retornar dados estatísticos da NBA dentro das propostas definidas, este notebook pretende alocar as etapas introdutórias para utilização de um banco de dados para armazenamento dos dados extraídos da API. Essa atuação, de alguma forma, será extremamente importante para propor um entendimento mais claro sobre como a solução como um todo pode ser integrada a outros sistemas e plataformas.

Para isso, será utilizado o **PostgreSQL** como SGBD principal e a biblioteca [`psycopg2`](https://pypi.org/project/psycopg2/) que permite a conexão entre Python e o SGBD. O comando para instalação da biblioteca diretamente no jupyter notebook é:

        !pip install psycopg2
        
<div align="center">
    <img src="https://cpl.thalesgroup.com/sites/default/files/content/paragraphs/intro/2020-03/postgresql-logo.png" height=500 width=300 alt="postgresql logo">
</div>

# Preparação do Ambiente

Após a [instalação](https://www.youtube.com/watch?v=wuAdsns3qMA&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=5) do [PostgreSQL](https://www.postgresql.org/download/) no sistema operacional utilizado e a configuração do ambiente a partir da [criação de novos usuários](https://www.youtube.com/watch?v=XqkGdFO-ENc&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=8) e definição de [políticas de senha](https://www.youtube.com/watch?v=HbAJQ1WuGRE&list=PLucm8g_ezqNoAkYKXN_zWupyH6hQCAwxY&index=7), é preciso criar um banco de dados. Essa criação pode ser realizada a partir do utilitário `psql` diretamente no prompt de comando ou então utilizando a interface gráfica proposta pelo [`pgAdmin4`](https://www.pgadmin.org/download/).

Para criação de um novo banco de dados no postgres, é preciso realizar o login utilizando um usuário com permissões de criação de bancos de dados e, no ambiente postgres, executar o comando `CREATE DATABASE <nome_database>;`. A imagem abaixo mostra os detalhes sobre esse procedimento a partir da criação do banco de dados _nbaflow_ utilizando o usuário _paninit_.

<div align="center">
    <br><img src="https://i.imgur.com/bCdtHLu.png" height=800 width=800 alt="postgresql logo">
</div>

# Conexão ao Banco de Dados

Uma vez criado o banco de dados, é possível realizar a conexão de um usuário a partir da função `connect()` do pacote `psycopg2`. A função `connect()` cria uma nova sessão do banco de dados e retorna uma instância da classe "conexão". Com esse novo objeto, é possível criar cursores para executar comandos SQL. Para executar a função `connect()` é preciso especificar, em um primeiro momento, parâmetros relacionados ao banco de dados do PostgreSQL como uma string de conexão.

In [8]:
# Adicionando diretório do ambiente virtual como uma variável de ambiente para importação dos pacotes
import sys
sys.path.append('/home/paninit/python-venvs/nbaflow-venv/lib/python3.8/site-packages')

# Biblioteca postgres
import psycopg2

# Importando bibliotecas
import os
from dotenv import load_dotenv, find_dotenv

In [18]:
# Lendo variáveis de ambiente
load_dotenv(find_dotenv())

# Definindo variáveis de diretório
HOST = '127.0.0.1'
DB_USER = 'paninit'
DB_NAME = 'nbaflow'

# Criando objeto de conexão
conn = psycopg2.connect(host=HOST,
                        database=DB_NAME,
                        user=DB_USER,
                        password=os.getenv('DB_PASSWORD'))

# Criando cursor e testando conexão
cur = conn.cursor()
print(f'Conexão realizada com sucesso no PostgreSQL versão:\n')
cur.execute('SELECT version()')
print(cur.fetchone()[0])
cur.close()

Conexão realizada com sucesso no PostgreSQL versão:

PostgreSQL 11.12 (Ubuntu 11.12-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0, 64-bit


No bloco acima, foi definido um exemplo de conexão com o banco de dados PostgreSQL utilizando credenciais passadas pelo próprio usuário. Como uma das informações requisitadas para a conexão é a senha do usuário, é preciso pensar em alternativas para mascarar essa informação e deixá-la inacessível para outras aplicações. Nesse primeiro momento, informações sensíveis (como a senha, por exemplo) podem ser armazenadas em um arquivo `.env` localizado no direorio do projeto, sendo sua leitura gerenciada pelas funções `find_dotenv()` e `load_dotenv()`. Dessa forma, é possível simular a inserção de variáveis como se fossem variáveis de ambiente do sistema operacional, permitindo assim sua posterior coleta utilizando a função `os.getenv()`.

Eventualmente, uma outra alternativa visa a criação de um [arquivo de conexão](https://www.postgresqltutorial.com/postgresql-python/connect/) para armazenar as informações sensíveis de conexão.

_PRÓXIMOS PASSOS_

    - Criação de tabela com o layout da base de gamelog extraída
    - Tentativa de criação dinâmica de tabela utilizando o schema extraído do próprio DataFrame
        * Encapsulamento de script
        * Alteração de parâmetros (nome de colunas e tipos primitivos)
        * Execução do script de criação, se aplicável
    - Proposta de ingestão de dados a partir de um DataFrame (INSERT INTO)